In [141]:
import os
from bs4 import BeautifulSoup

import re

In [142]:
def extract_numbers_from_html(html_file):
    with open(html_file, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')
        # print(soup.prettify())

        a_tags = soup.find_all('a')
        file_names = []
        for a_tag in a_tags:
            file_names.append(a_tag.get_text(strip=True).split(':')[0])
        # print(file_names)

        
        # Find all tags with td
        td_tags = soup.find_all('td')
        clone_pairs = 0
        clone_classes = 0
        for td_tag in td_tags:
            if'Clone pairs' in td_tag.get_text(strip=True) :
                clone_pairs = re.search(r'\d+', td_tag.get_text(strip=True)).group()
            if 'Clone classes' in td_tag.get_text(strip=True):
                clone_classes = re.search(r'\d+', td_tag.get_text(strip=True)).group()
        return clone_pairs, clone_classes, '#'.join(file_names)


In [143]:
def traverse_and_extract_numbers(root_folder):
    numbers = []
    for foldername, subfolders, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.endswith('.html'):
                html_file_path = os.path.join(foldername, filename)
                clone_pairs, clone_classes, file_names = extract_numbers_from_html(html_file_path)
                numbers.append((html_file_path, clone_pairs, clone_classes, file_names))
    return numbers



In [144]:
root_folder = './Output'
result = traverse_and_extract_numbers(root_folder)

data = []
model = 'GPT-3.5'
for filename, clone_pairs, clone_classes,file_names in result:
    id = filename.split('/')[2]
    clone_type = filename.split('/')[3].split('.')[0]

    # print(f'{id},{clone_type},{clone_pairs},{clone_classes}')
    data.append((model,id, clone_type, clone_pairs, clone_classes,file_names))


In [145]:
import pandas as pd
df = pd.DataFrame(data, columns=['model', 'id', 'clone_type', 'clone_pairs', 'clone_classes', 'file_names'])
df.to_csv('result.csv', index=False)